In [18]:
import pandas as pd
import geocoder
from math import radians, cos, sin, asin, sqrt
bing_key = ""

In [9]:
df = pd.DataFrame({'A_address': ['Times Square',], 
              'A_city': ['Manhattan', ], 
              'A_state': ['NY', ], 
              'B_address': ['Walt Disney World Resort', ], 
              'B_city': ['Orlando',], 
              'B_state':['FL']}, index = range(1))
df

,A_address,A_city,A_state,B_address,B_city,B_state
0,Times Square,Manhattan,NY,Walt Disney World Resort,Orlando,FL


In [35]:
df.to_html().replace("<th>", '<th scope="row">').replace(
    '"dataframe', '"table-responsive table-bordered table-hover table-striped"').replace("\n", "")

'<table border="1" class="table table-striped"">  <thead>    <tr style="text-align: right;">      <th scope="row"></th>      <th scope="row">A_address</th>      <th scope="row">A_city</th>      <th scope="row">A_state</th>      <th scope="row">B_address</th>      <th scope="row">B_city</th>      <th scope="row">B_state</th>      <th scope="row">A_full_address</th>      <th scope="row">B_full_address</th>      <th scope="row">A_lng</th>      <th scope="row">A_lat</th>      <th scope="row">B_lng</th>      <th scope="row">B_lat</th>      <th scope="row">distance_in_miles</th>    </tr>  </thead>  <tbody>    <tr>      <th scope="row">0</th>      <td>Times Square</td>      <td>Manhattan</td>      <td>NY</td>      <td>Walt Disney World Resort</td>      <td>Orlando</td>      <td>FL</td>      <td>Times Square, Manhattan NY</td>      <td>Walt Disney World Resort, Orlando FL</td>      <td>-73.966248</td>      <td>40.783436</td>      <td>-81.582626</td>      <td>28.403811</td>      <td>957.218796<

In [10]:
# get full address
df['A_full_address'] = ["%s, %s %s"%(addr, city, state) for addr, city, state in zip(df.A_address, 
                                                             df.A_city, 
                                                             df.A_state)]
df['B_full_address'] = ["%s, %s %s"%(addr, city, state) for addr, city, state in zip(df.B_address, 
                                                             df.B_city, 
                                                             df.B_state)]

In [11]:
for i in df.index:
    g = geocoder.bing(df.loc[i, 'A_full_address'], key = bing_key)
    df.loc[i, 'A_lng'] = g.lng
    df.loc[i, 'A_lat'] = g.lat  

In [39]:
 df_a = df.loc[:, [x for x in df.columns if x[0] == 'A']]
## de-duplicate
 df_a = df_a.groupby('A_full_address').first().reset_index(drop=False)

In [40]:
df_a

,A_full_address,A_address,A_city,A_state,A_lng,A_lat
0,"Times Square, Manhattan NY",Times Square,Manhattan,NY,-73.966248,40.783436


In [14]:
def get_loc(obj): 
    g = geocoder.bing(obj, key = bing_key)
    return g.lng, g.lat
df[['B_lng', 'B_lat']] = df.apply(lambda obj: pd.Series(
    dict(zip(['B_lng', 'B_lat'],
        get_loc(obj['B_full_address'])))), axis = 1)

In [21]:
df.to_html()

'<table border="1" class="dataframe">\n  <thead>\n    <tr style="text-align: right;">\n      <th></th>\n      <th>A_address</th>\n      <th>A_city</th>\n      <th>A_state</th>\n      <th>B_address</th>\n      <th>B_city</th>\n      <th>B_state</th>\n      <th>A_full_address</th>\n      <th>B_full_address</th>\n      <th>A_lng</th>\n      <th>A_lat</th>\n      <th>B_lng</th>\n      <th>B_lat</th>\n      <th>distance_in_miles</th>\n    </tr>\n  </thead>\n  <tbody>\n    <tr>\n      <th>0</th>\n      <td>Times Square</td>\n      <td>Manhattan</td>\n      <td>NY</td>\n      <td>Walt Disney World Resort</td>\n      <td>Orlando</td>\n      <td>FL</td>\n      <td>Times Square, Manhattan NY</td>\n      <td>Walt Disney World Resort, Orlando FL</td>\n      <td>-73.966248</td>\n      <td>40.783436</td>\n      <td>-81.582626</td>\n      <td>28.403811</td>\n      <td>957.218796</td>\n    </tr>\n  </tbody>\n</table>'

In [16]:
def haversine(lon1, lat1, lon2, lat2):
    """
    Calculate the great circle distance between two points 
    on the earth (specified in decimal degrees)
    """
    # convert decimal degrees to radians 
    lon1, lat1, lon2, lat2 = map(radians, [lon1, lat1, lon2, lat2])
    # haversine formula 
    dlon = lon2 - lon1 
    dlat = lat2 - lat1 
    a = sin(dlat/2)**2 + cos(lat1) * cos(lat2) * sin(dlon/2)**2
    c = 2 * asin(sqrt(a)) 
    km = 6367 * c
    mile = km * 0.621371
    return mile

In [19]:
df['distance_in_miles'] = df.apply(lambda obj: haversine(*obj[['A_lng', 'A_lat', 'B_lng', 'B_lat']]), axis = 1)

In [20]:
df

,A_address,A_city,A_state,B_address,B_city,B_state,A_full_address,B_full_address,A_lng,A_lat,B_lng,B_lat,distance_in_miles
0,Times Square,Manhattan,NY,Walt Disney World Resort,Orlando,FL,"Times Square, Manhattan NY","Walt Disney World Resort, Orlando FL",-73.966248,40.783436,-81.582626,28.403811,957.218796
